<a href="https://colab.research.google.com/github/aglucaci/longarc/blob/main/LONGARC_SCREENER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [15]:
!pip install finvizfinance
!pip install yfinance

In [16]:
from finvizfinance.screener.overview import Overview
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime, timedelta
import time

# Declares

In [17]:
filters_dict = {
    'Debt/Equity': 'Under 1',
    'EPS growthpast 5 years': 'Over 15%',
    'Price/Free Cash Flow': 'Under 50',
    'Return on Assets': 'Positive (>0%)',
    'Return on Equity': 'Over +15%',
    'Return on Investment': 'Over +15%',
    '52-Week High/Low': '0-10% above Low'
    #'52-Week High/Low': 'New Low'
}


"""
ValueError: Invalid filter 'EPS growth past 5 years'. Possible filter: ['Exchange', 'Index', 'Sector', 'Industry', 'Country', 'Market Cap.', 'P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'Price/Cash', 'Price/Free Cash Flow', 'EPS growththis year', 'EPS growthnext year', 'EPS growthpast 5 years', 'EPS growthnext 5 years', 'Sales growthpast 5 years', 'EPS growthqtr over qtr', 'Sales growthqtr over qtr', 'Dividend Yield', 'Return on Assets', 'Return on Equity', 'Return on Investment', 'Current Ratio', 'Quick Ratio', 'LT Debt/Equity', 'Debt/Equity', 'Gross Margin', 'Operating Margin', 'Net Profit Margin', 'Payout Ratio', 'InsiderOwnership', 'InsiderTransactions', 'InstitutionalOwnership', 'InstitutionalTransactions', 'Float Short', 'Analyst Recom.', 'Option/Short', 'Earnings Date', 'Performance', 'Performance 2', 'Volatility', 'RSI (14)', 'Gap', '20-Day Simple Moving Average', '50-Day Simple Moving Average', '200-Day Simple Moving Average', 'Change', 'Change from Open', '20-Day High/Low', '50-Day High/Low', '52-Week High/Low', 'Pattern', 'Candlestick', 'Beta', 'Average True Range', 'Average Volume', 'Relative Volume', 'Current Volume', 'Price', 'Target Price', 'IPO Date', 'Shares Outstanding', 'Float']
"""


"\nValueError: Invalid filter 'EPS growth past 5 years'. Possible filter: ['Exchange', 'Index', 'Sector', 'Industry', 'Country', 'Market Cap.', 'P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'Price/Cash', 'Price/Free Cash Flow', 'EPS growththis year', 'EPS growthnext year', 'EPS growthpast 5 years', 'EPS growthnext 5 years', 'Sales growthpast 5 years', 'EPS growthqtr over qtr', 'Sales growthqtr over qtr', 'Dividend Yield', 'Return on Assets', 'Return on Equity', 'Return on Investment', 'Current Ratio', 'Quick Ratio', 'LT Debt/Equity', 'Debt/Equity', 'Gross Margin', 'Operating Margin', 'Net Profit Margin', 'Payout Ratio', 'InsiderOwnership', 'InsiderTransactions', 'InstitutionalOwnership', 'InstitutionalTransactions', 'Float Short', 'Analyst Recom.', 'Option/Short', 'Earnings Date', 'Performance', 'Performance 2', 'Volatility', 'RSI (14)', 'Gap', '20-Day Simple Moving Average', '50-Day Simple Moving Average', '200-Day Simple Moving Average', 'Change', 'Change from Open', '20-Day High/Low', 

# Helper functions

In [18]:
def get_finviz_tickers(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    tickers = []
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.find_all('a', class_='screener-link-primary')
        page_tickers = [row.text for row in rows]
        if not page_tickers:
            break
        tickers.extend(page_tickers)

        # Check for next page
        next_link = soup.find('a', text='next')
        if not next_link:
            break
        url = "https://finviz.com/" + next_link['href']
    return list(set(tickers))  # remove duplicates

def calculate_cagr(ticker):
    try:
        stock = yf.Ticker(ticker)
        end_date = datetime.today()
        start_date = end_date - timedelta(days=365 * 10)

        hist = stock.history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        if len(hist) == 0:
            return None

        start_price = hist['Close'][0]
        end_price = hist['Close'][-1]

        cagr = (end_price / start_price) ** (1 / 10) - 1
        return round(cagr * 100, 2)  # Return CAGR as percentage
    except Exception as e:
        print(f"Error with {ticker}: {e}")
        return None

# Main

In [19]:
foverview = Overview()
foverview.set_filter(filters_dict=filters_dict)
df = foverview.screener_view()

# CAGR Analysis
for index, row in df.iterrows():
    #time.sleep(60)
    ticker = row['Ticker']
    print(ticker, "calculating CAGR....")
    cagr = calculate_cagr(ticker)
    if cagr is not None:
        df.at[index, '10yr_CAGR'] = cagr # 10 year CAGR
    # end if
    time.sleep(1)
# end for

# Dividend Yield Analysis
for index, row in df.iterrows():
    time.sleep(1)
    ticker = row['Ticker']
    ticker = yf.Ticker(ticker)
    try:
        dividend_yield = ticker.info.get('dividendYield')
    except:
        dividend_yield = None
    print(ticker, "calculating dividend yield....")

    if dividend_yield is not None:
        df.at[index, 'Dividend Yield'] = dividend_yield
    # end if
# end for

BMI calculating CAGR....


/tmp/ipython-input-370727753.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_price = hist['Close'][0]
/tmp/ipython-input-370727753.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_price = hist['Close'][-1]


CRVL calculating CAGR....


/tmp/ipython-input-370727753.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_price = hist['Close'][0]
/tmp/ipython-input-370727753.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_price = hist['Close'][-1]


CTAS calculating CAGR....


...

V calculating CAGR....


/tmp/ipython-input-370727753.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_price = hist['Close'][0]
/tmp/ipython-input-370727753.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_price = hist['Close'][-1]


YELP calculating CAGR....


/tmp/ipython-input-370727753.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_price = hist['Close'][0]
/tmp/ipython-input-370727753.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_price = hist['Close'][-1]


yfinance.Ticker object <BMI> calculating dividend yield....
yfinance.Ticker object <CRVL> calculating dividend yield....
yfinance.Ticker object <CTAS> calculating dividend yield....
yfinance.Ticker object <DOCS> calculating dividend yield....
yfinance.Ticker object <FUBO> calculating dividend yield....
yfinance.Ticker object <IDT> calculating dividend yield....
yfinance.Ticker object <INMD> calculating dividend yield....
yfinance.Ticker object <NFLX> calculating dividend yield....
yfinance.Ticker object <ODD> calculating dividend yield....
yfinance.Ticker object <OSPN> calculating dividend yield....
yfinance.Ticker object <PAYC> calculating dividend yield....
yfinance.Ticker object <PCTY> calculating dividend yield....
yfinance.Ticker object <PGR> calculating dividend yield....
yfinance.Ticker object <RACE> calculating dividend yield....
yfinance.Ticker object <RLI> calculating dividend yield....
yfinance.Ticker object <TTD> calculating dividend yield....
yfinance.Ticker object <V> cal

In [28]:
df.to_csv('output.csv')

In [20]:
#print("Current date and time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Current date and time: 2026-01-25 22:57:44


In [24]:
#print(df.head())

  Ticker             Company                  Sector  \
0    BMI    Badger Meter Inc              Technology   
1   CRVL         Corvel Corp               Financial   
2   CTAS  Cintas Corporation             Industrials   
3   DOCS        Doximity Inc              Healthcare   
4   FUBO          fuboTV Inc  Communication Services   

                             Industry Country    Market Cap    P/E   Price  \
0  Scientific & Technical Instruments     USA  5.000000e+09  36.12  169.52   
1                   Insurance Brokers     USA  3.580000e+09  34.44   69.88   
2         Specialty Business Services     USA  7.716000e+10  41.71  192.96   
3         Health Information Services     USA  7.650000e+09  32.41   40.63   
4                        Broadcasting     USA  8.499600e+08   6.92    2.48   

   Change     Volume  10yr_CAGR  Dividend Yield  
0 -0.0152   211788.0      21.48            0.87  
1 -0.0165   124190.0      17.04             NaN  
2 -0.0006  1403970.0      26.23            0

In [27]:
#df_flt = df[df["10yr_CAGR"] > 15]
#df_flt

In [26]:

"""
for index, row in df_flt.iterrows():
    ticker = row['Ticker']
    # https://www.gurufocus.com/stock/BLD/dcf
    _url = "https://www.gurufocus.com/stock/" + ticker + "/dcf"
    print(ticker, "examining DCF Margin of Safety....", _url)
    #df_flt.at[index, 'DCF Margin of Safety'] = _url
    df_flt.loc[index, 'DCF Margin of Safety'] = _url
    time.sleep(1)
# end for

df_flt.to_csv('output.csv')
"""


'\nfor index, row in df_flt.iterrows():\n    ticker = row[\'Ticker\']\n    # https://www.gurufocus.com/stock/BLD/dcf\n    _url = "https://www.gurufocus.com/stock/" + ticker + "/dcf"\n    print(ticker, "examining DCF Margin of Safety....", _url)\n    #df_flt.at[index, \'DCF Margin of Safety\'] = _url\n    df_flt.loc[index, \'DCF Margin of Safety\'] = _url\n    time.sleep(1)\n# end for\n\ndf_flt.to_csv(\'output.csv\')\n'

# Visualizations / 10yr Charts

In [25]:
"""
import matplotlib.pyplot as plt
import os

os.makedirs("images", exist_ok=True)

for index, row in df.iterrows():
  data = yf.download(row['Ticker'], period="10y")
  data['Close'].plot(title=f"{row['Ticker']} Closing Price")
  plt.savefig(f"images/{row['Ticker']}.png")
  #plt.show()
"""

'\nimport matplotlib.pyplot as plt\nimport os\n\nos.makedirs("images", exist_ok=True)\n\nfor index, row in df.iterrows():\n  data = yf.download(row[\'Ticker\'], period="10y")\n  data[\'Close\'].plot(title=f"{row[\'Ticker\']} Closing Price")\n  plt.savefig(f"images/{row[\'Ticker\']}.png")\n  #plt.show()\n'